# Assignment 20: Polymorphism, Inheritance, and Overriding #

### Goals for this Assignment ###

By the time you have completed this assignment, you should be able to:

- Define functions which use _polymorphism_ to transparently call different methods with the same name
- Define classes which inherit from other classes
- Define constructors which call the parent class' constructor
- Override inherited methods in a child class

## Step 1: Define a Function Which Uses Polymorphism ##

### Background: Polymorphism in Python ###

At this point, you are used to calling methods on objects, and even defining your own methods on objects.
There is a bit of functionality here which effectively comes for free in Python, which is easy to overlook.
Specifically, when you call a method in Python, Python only checks if the given object has a given method with the same name and correct number of arguments.
Python doesn't check anything deeper.
Because of this, you can substitute in different objects as the target of the call, as long as all the objects share a method with the same name and number of parameters.

As an example, we will revisit some various shape-related classes from assignment 7 in the next cell below.

In [3]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
    def area(self):
        return 3.14 * (self.radius ** 2)
    def perimeter(self):
        return 2 * 3.14 * self.radius

class Square:
    def __init__(self, side):
        self.side = side
    def area(self):
        return self.side ** 2
    def perimeter(self):
        return 4 * self.side

def print_area_perimeter(shape):
    print(shape.area())
    print(shape.perimeter())

circle = Circle(3)
square = Square(5)

print_area_perimeter(circle)
print()
print_area_perimeter(square)

28.26
18.84

25
20


Importantly, in the `print_area_perimeter` function, the underlying type of `shape` is actually different for the two different calls later in the code.
In the first call, `shape` is of type `Circle`, but in the second call, `shape` is of type `Square`.
Despite the fact that the type of `shape` is different, and despite the fact that `print_area_perimeter` calls methods on the given shape, this code works problem-free.
This is because both `Circle` and `Square` define the `area` and `perimeter` methods, which each take no arguments.
This is the only real _contract_ (meaning, software expectations) that `print_area_perimeter` puts on `shape`: the `shape` needs to define these `area` and `perimeter` methods, and if so, then `print_area_perimeter` will work correctly, no matter the underlying type of `shape`.
This ability to substitute types is often referred to as _polymorphism_, which comes from a Greek word meaning _many forms_.
That is, the underlying object can have different types.
This ability is also sometimes called _duck typing_, which is from the adage that if "something looks like a duck, swims like a duck, and quacks like a duck, then it is probably a duck".
Basically, as long as an object looks enough like a duck, then it will work as a duck.

> Slight pedantic aside/rant, which can be completely skipped: this is specifically exhibiting _ad-hoc polymorphism_, which means that the actual choice of method called can be delayed until runtime.
> The term "polymorphism" by itself is not well-defined, as there are other kinds of polymorphism which are unrelated to how this code works, such as subtyping polymorphism, parametric polymorphism, and row polymorphism.
> All of these different kinds of polymorphism share the property of allowing you to reuse code for different kinds of things, but the particularities are wildly different.
> However, outside of programming language design, this is generally not well-understood, and people commonly use the term "polymorphism" to refer to both ad-hoc polymorphism and subtyping polymorphism.
> While we will discuss subclasses, we will not discuss subtyping polymorphism, because strictly speaking, Python doesn't have it, as it is subsumed by Python's dynamic typing.
> Furthermore, the term "duck typing" is also not particularly well-defined once you get into all the details, so I'll be avoiding the term.
> I will use the unqualified term "polymorphism" from this point forward, as this is consistent with how this information is usually presented on the Internet or in textbooks.

The take-home point with this is that you can use polymorphism to reuse code (specifically our `print_area_permeter` function above) by passing in different kinds of objects, as long as all the objects you pass define the methods you call.

You've actually been using a predefined polymorphic function for awhile now: `len`.
As a recap, `len` can be used to get the length of strings, lists, and dictionaries, as with:

In [4]:
print(len("foobar"))
print(len(["alpha", "beta"]))
print(len({ "gamma" : "delta" }))

6
2
1


Internally, `len` ends up calling the given object's `__len__` method, which takes no arguments.
We can see this with a custom class, like so:

In [5]:
class CustomLength:
    def __init__(self, expected_len):
        self.expected_len = expected_len
    def __len__(self):
        return self.expected_len

c1 = CustomLength(3)
c2 = CustomLength(6)
print(len(c1))
print(len(c2))

3
6


As shown with `CustomLength`, `len` internally is calling the `__len__` method on the passed object, in order to determine the actual length of the object.

### Try this Yourself ###

In the next cell, there are two classes predefined which define a `do_operation` method.
Note that even though these classes do not define constructors, Python effectively gives them the following constructor:

```python
def __init__(self):
    pass # needed to say the body is otherwise empty
```

That is, the constructor exists, but it doesn't do anything particularly useful.

`do_operation` takes two integers, and will return an integer.
The implementations of `do_operation` are different for the two different classes.

Define a function named `test_operation` which:

- Takes an object defining a `do_operation` method with the aforementioned expected parameters and return value
- Prints out the result of calling `do_operation` with `3` and `7`
- Prints out the result of calling `do_operation` with `2` and `9`

Define `test_operation` in the next cell.
Leave the calls in place in order to test your code.
The expected output to be printed out is shown in the comments.

In [6]:
# Define your test_operation function here.  Leave the calls at the end for testing

def test_operation(param):
    print(param.do_operation(3, 7))
    print(param.do_operation(2, 9))
    

class Add:
    def do_operation(self, x, y):
        return x + y

class Multiply:
    def do_operation(self, x, y):
        return x * y

add = Add();
mult = Multiply()

test_operation(add)
# above statement should print:
# 10
# 11

print()
test_operation(mult)
# above statement should print:
# 21
# 18

10
11

21
18


## Step 2: Define a Class that Inherits Functionality from Another Class ##

### Background: Class Inheritance ###

Let's revisit the `print_area_perimeter` function from the prior step, repeated below for convenience:

```python
def print_area_perimeter(shape):
    print(shape.area())
    print(shape.perimeter())
```

While this code works, it's arguably not ideal from the standpoint of how the code is structured.
If we read the code, we can see that it is intended to work with shapes, but there is nothing in the larger code structure which indicates this.
Functions sort of float in the ether, and are only connected to whatever module they are defined in.
Sometimes this is appropriate, but many times it isn't.
In this case, since `print_area_perimeter` is really intended to work with shapes, it makes more sense to define it directly on our shape classes.
This is illustrated in the cell below, which redefines our `Circle` and `Square` classes to make `print_area_perimeter` a method instead:

In [6]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
    def area(self):
        return 3.14 * (self.radius ** 2)
    def perimeter(self):
        return 2 * 3.14 * self.radius
    def print_area_perimeter(self):
        print(self.area())
        print(self.perimeter())

class Square:
    def __init__(self, side):
        self.side = side
    def area(self):
        return self.side ** 2
    def perimeter(self):
        return 4 * self.side
    def print_area_perimeter(self):
        print(self.area())
        print(self.perimeter())

circle = Circle(3)
square = Square(5)

circle.print_area_perimeter()
print()
square.print_area_perimeter()

28.26
18.84

25
20


As shown, we now call `print_area_perimeter` as a method on `circle` and `square`, which directly connects this to the definition of those classes.
Now, from the code structure itself, we know that `print_area_perimeter` is to be used with these classes, without knowing any other details.
This also means that instead of calling `print_area_perimeter(circle)`, we call `circle.print_area_perimeter()`, and so on.

However, as written, there is an enormous downside here: we completely duplicated the definition of `print_area_perimeter` once for `Circle`, and another time for `Square`.
This is referred to as _copy/paste programming_, which has some major downsides:

- It can take time to copy/paste code into all the places it needs to go
- It can bloat the side of the code, making it take longer to read, understand, and especially debug
- (Most important in my opinion) if there are any bugs in the code you copy/paste (and you usually won't know for sure), you end up duplicating the bug every time you copy/paste the code.  This can easily lead to situations where you find a bug, fix it, and then later discover that you needed to fix the same bug somewhere else because the original buggy code has been copied elsewhere.

Ideally, we only want to define our `print_area_perimeter` method once, but nonetheless have this method be used in both `Circle` and `Square`.
Python allows us to reuse code in this fashion without copy/pasting via _inheritance_.
The idea with inheritance is that you can define a class which contains any code you want to share.
We will refer to this class as the _base class_, or _parent class_.
You then define classes which _inherit_ from the base class, and in so doing, get all the code defined on that base class.
These classes which inherit from the base class are known as _subclasses_, or _child classes_.
This means we only need to define any shared code once, specifically in the base class.
Through inheritance, that code is automatically available in subclasses.
This addresses all the aforementioned issues with copy/paste coding, because the shared code is only ever defined once (in the base class).

This is best shown through example.
The following cell redefines our `Circle` and `Square` classes.
Instead of duplicating the `print_area_perimeter` method, they instead inherit from a common base class named `Shape`.

In [7]:
class Shape:
    def print_area_perimeter(self):
        print(self.area())
        print(self.perimeter())

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    def area(self):
        return 3.14 * (self.radius ** 2)
    def perimeter(self):
        return 2 * 3.14 * self.radius

class Square(Shape):
    def __init__(self, side):
        self.side = side
    def area(self):
        return self.side ** 2
    def perimeter(self):
        return 4 * self.side

circle = Circle(3)
square = Square(5)

circle.print_area_perimeter()
print()
square.print_area_perimeter()

28.26
18.84

25
20


The actual inheritance part happens on the same line as `class` is used for `Circle` and `Square`.
Specifically, before the `:`, you can put a class name in parentheses.
This such a class name is provided, the defined class will inherit from the provided class.
In other words:

```python
class Square(Shape):
```

...says that we are defining the `Square` class, and `Square` is inheriting from the `Shape` class.
This is why we are later able to call `print_area_perimeter` on `square` (which is an instance of `Square`): even though `Square` itself doesn't define `print_area_perimeter`, `Square` inherits from `Shape`, and `Shape` _does_ define `print_area_perimeter`.
As such, `Square` inherits the definition of `print_area_perimeter` from `Shape`, only by saying that `Square` inherits from `Shape`.

### Try this Yourself ###

The prior step required you to define the `test_operation` function, which would take either an `Add` or `Multiply` instance.
Redefine this code below to make `test_operation` instead a method defined on a base class, and have `Add` and `Multiply` inherit from that base class.
You will need to redefine `Add` and `Multiply` themselves to make this work, though you should only need to change the line that uses `class`.

Define your code in the next cell.
Leave the existing code in place in order to test your code.

In [2]:
# Define your code here.  Leave the code below in order to test what you write.
       
class Operation:
    def test_operation(self):
        print(self.do_operation(3, 7))
        print(self.do_operation(2, 9))

class Add(Operation):
    def do_operation(self, x, y):
        return x + y

class Multiply(Operation):
    def do_operation(self, x, y):
        return x * y

add = Add()
mult = Multiply()

add.test_operation()
# above statement should print:
# 10
# 11

print()
mult.test_operation()
# above statement should print:
# 21
# 18

10
11

21
18


## Step 3: Define a Subclass that Calls the Parent Class' Constructor ##

### Background: Inheritance with Constructors ###

In the code we've worked with so far, we have not defined any constructors.
Specifically, we have relied upon the fact that Python will automatically define a constructor for us which will not take any arguments and not do anything particularly useful.
However, this is not always appropriate.
For example, say we modify the `Shape` class from the prior example, so that we have a constructor which:

- Takes the name of the particular kind of shape
- Saves that name
- Prints out that a shape with this name has been created

We will also add a `get_name` method, which will get the name of this shape.
All of this is shown in the cell below, along with our `Circle` and `Square` classes:

In [3]:
class Shape:
    def __init__(self, name):
        self.name = name
        print(f"Created a {name}")
    def print_area_perimeter(self):
        print(self.area())
        print(self.perimeter())
    def get_name(self):
        return self.name

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    def area(self):
        return 3.14 * (self.radius ** 2)
    def perimeter(self):
        return 2 * 3.14 * self.radius

class Square(Shape):
    def __init__(self, side):
        self.side = side
    def area(self):
        return self.side ** 2
    def perimeter(self):
        return 4 * self.side

circle = Circle(3)
square = Square(5)

circle.print_area_perimeter()
print()
square.print_area_perimeter()

print()
print(circle.get_name())
print(square.get_name())

28.26
18.84

25
20



AttributeError: 'Circle' object has no attribute 'name'

If you run this cell, everything will work as normal until the `circle.get_name` method is called.
At that point, `get_name` attempts to access the `name` field, but we instead get an `AttributeError`, indicating that there is no `name` field on the circle.
This is because we **never ran `Shape`'s constructor**, which is ultimately what defines the `name` field.

This shows that constructors are in a sort of weird space when it comes to inheritance.
Strictly speaking, constructors are not normal methods, and are therefore not inherited.
If a parent class' constructor does some work (as `Shape` does above), it's up to the programmer to ensure that this work is performed in subclass constructors.
We could do this by copy/pasting what the `Shape` constructor does (and in so reintroducing all the problems with copy/paste coding), but a better approach is to instead explicitly call the parent class' constructor in the subclass, like so:

In [10]:
class Shape:
    def __init__(self, name):
        self.name = name
        print(f"Created a {name}")
    def print_area_perimeter(self):
        print(self.area())
        print(self.perimeter())
    def get_name(self):
        return self.name

class Circle(Shape):
    def __init__(self, radius):
        Shape.__init__(self, "circle")
        self.radius = radius
    def area(self):
        return 3.14 * (self.radius ** 2)
    def perimeter(self):
        return 2 * 3.14 * self.radius

class Square(Shape):
    def __init__(self, side):
        Shape.__init__(self, "square")
        self.side = side
    def area(self):
        return self.side ** 2
    def perimeter(self):
        return 4 * self.side

circle = Circle(3)
square = Square(5)

circle.print_area_perimeter()
print()
square.print_area_perimeter()

print()
print(circle.get_name())
print(square.get_name())

Created a circle
Created a square
28.26
18.84

25
20

circle
square


In the constructors of `Circle` and `Square`, we now call `Shape`'s constructor directly via `Shape.__init__`.
We need to pass along the reference to the object being created `self`, and since `Shape` expects the name of the kind of shape this is, a string representing the sort of shape we are dealing with.
After calling the parent class' constructor, the subclass' constructor is free to do whatever work is necessary for initializing the subclass; the `Circle` and `Square` constructors initialize the `radius` and `side` fields, respectively, after calling `Shape`'s constructor.

> For those coming from most other languages, you may be used to using `super` to access the parent class' constructor.
> You may also have seen that Python _does_ have `super`, and that with some syntactic differences, it can do the same things that the `super` you're used to does.
> Lutz chapter 28, page 860 has a more detailed explanation of this, but `super` in Python is surprisingly complex, and its behavior has changed over time as Python has gone through different versions.
> Part of this is because Python allows for multiple inheritance, wherein a class is permitted to inherit from more than one class.
> In this course, we will only ever deal with single inheritance, simplifying this problem.
> Calling the superclass' constructor directly requires far less explanation and is a lot more predictable, so we will stick to this approach in this course.

### Try this Yourself ###

In the next cell, a class named `Animal` is defined, where animals:

- Have a constructor, taking a name.
- Have a `name` field, initialized by the constructor.
- Have a `get_name` method, which returns the name.

Subclasses of `Animal` are intended to define the `speak` method, which prints out a string corresponding to the particular kind of animal.

In the next cell, define `Dog` and `Cat` classes which inherit from `Animal`, and take a name in their constructor.
From there, you should call `Animal`'s constructor to set the name.
`Dog`'s `speak` method should print `"woof"`, and `Cat`'s `speak` method should print `"meow"`.
Leave all the existing code in place in order to test your implementation.

In [6]:
class Animal:
    def __init__(self, name):
        self.name = name
    def get_name(self):
        return self.name
        
# Define your Dog and Cat classes below.
# Be sure to call Animal's constructor in the constructors for Dog and Cat.

       
class Dog(Animal):
    def __init__(self, name):
        Animal.__init__(self,name)   
    def speak(self):
        print("woof")

class Cat(Animal):
    def __init__(self, name):
        Animal.__init__(self,name)   
    def speak(self):
        print("meow")


# Leave the code below in order to test your implementations.
d1 = Dog("Rover")
c1 = Cat("Otis")

print(d1.name) # should print "Rover"
print(d1.get_name()) # should print "Rover"
d1.speak() # should print "woof"

print()

print(c1.name) # should print "Otis"
print(c1.get_name()) # should print "Otis"
c1.speak() # should print "meow"

Rover
Rover
woof

Otis
Otis
meow


The actual inheritance part happens on the same line as `class` is used for `Circle` and `Square`.
Specifically, before the `:`, you can put a class name in parentheses.
This such a class name is provided, the defined class will inherit from the provided class.
In other words:

```python
class Square(Shape):
```

...says that we are defining the `Square` class, and `Square` is inheriting from the `Shape` class.
This is why we are later able to call `print_area_perimeter` on `square` (which is an instance of `Square`): even though `Square` itself doesn't define `print_area_perimeter`, `Square` inherits from `Shape`, and `Shape` _does_ define `print_area_perimeter`.
As such, `Square` inherits the definition of `print_area_perimeter` from `Shape`, only by saying that `Square` inherits from `Shape`.

### Try this Yourself ###

The prior step required you to define the `test_operation` function, which would take either an `Add` or `Multiply` instance.
Redefine this code below to make `test_operation` instead a method defined on a base class, and have `Add` and `Multiply` inherit from that base class.
You will need to redefine `Add` and `Multiply` themselves to make this work, though you should only need to change the line that uses `class`.

Define your code in the next cell.
Leave the existing code in place in order to test your code.

## Step 4: Override an Inherited Method ##

### Background: Overriding Methods ###

Sometimes you may inherit a method which you want to define yourself, as opposed to using the parent class' definition.
For example, let's consider another polymorphic function you've been using for awhile now: `str`.
As a reminder, `str` can be used to get the string representation of a given object, as with:

In [7]:
print(str("foo")) # already a string - gets the same string
print(str(3.14)) # goes from a float to a string
print(str(17)) # goes from an int to a string
print(str(["alpha", "beta"])) # goes from a list to a string
print(str({ "gamma" : "delta" })) # goes from a dict to a string

foo
3.14
17
['alpha', 'beta']
{'gamma': 'delta'}


The `str` function actually works in a similar manner as the `len` function.
Specifically, `str` will internally call the `__str__` method on a given object.
We can see this happening with the class below:

In [8]:
class CustomStr:
    def __str__(self):
        return "custom string"

c = CustomStr()
print(str(c))

custom string


As shown, `str(c)` ends up returning `"custom string"`, showing that `str` is internally calling `c`'s `__str__` method.

In Python, even if you don't provide a class to inherit from, you automatically will inherit from a built-in class named `object`.
`object` comes predefined with a `__str__` method for us, meaning we don't need to define our own.
This means we can call `str` on an object even if the class doesn't define its own `__str__` method, because you'll inherit a definition of `__str__` from `object`.
This is shown in the cell below:

In [9]:
class Empty:
    pass

e = Empty()
print(str(e))
print(e.__str__()) # directly does what `str` does internally

As shown, there _is_ a `__str__` method we are inheriting.
The inherited `__str__` shows the name of the module the class was defined in (`__main__`), as well as the name of the class the object was created from (`Empty`).
The inherited `__str__` also shows where the object itself is (likely) allocated in memory, represented with a _hexadecimal_ (base-16) number; hexadecimal numbers customarily start with `0x` to indicate that they are hexadecimal.
The specific location (and corresponding hexadecimal number) will likely be different each time you run the above cell, since each run of the cell allocates a new `Empty` object.

Whether or not this is a _good_ implementation of `__str__` for your class is entirely up to you.
However, generally, if you want to be able to get string representations of your object, this is not a preferred representation.
For example, if `int` used this representation, instead of `str(123)` returning `"123"`, it'd instead return something more like `<__some_module__.int object at 0x83d497c>`.
While this would be _a_ string representation of an integer, this is probably not _the expected_ string representation of an integer.

If we want a different string representation of whatever object we are dealing with, we can _override_ the inherited definition of `__str__` to return whatever string we want.
In fact, `CustomStr` did exactly this: `CustomStr` overrode the inherited definition of `__str__` from `object`.
This means that any instances of `CustomStr` will instead call the overridden definition of `__str__`, instead of the original `__str__` inherited from `object`.

### Try this Yourself ###

The next cell defines the following:

- A `Base` class, which defines a `print_me` method.  This prints `"base"` when called.
- A `Sub` class, which inherits from `Base`, and in so doing inherits `Base`'s `print_me` method.

Modify `Sub` to override the `print_me` method in `Sub` so that it instead prints `"sub"`.
Leave the calls at the end of the code in order to test your implementation.

In [13]:
class Base:
    def print_me(self):
        print("base")

class Sub(Base):
    #pass # you will likely need to delete this line
    def print_me(self):
        print("sub")

b = Base()
s = Sub()

b.print_me() # should print "base"
s.print_me() # should print "sub"

base
sub


## Step 5: Submit via Canvas ##

Be sure to **save your work**, then log into [Canvas](https://canvas.csun.edu/).  Go to the COMP 502 course, and click "Assignments" on the left pane.  From there, click "Assignment 20".  From there, you can upload the `20_inheritance_and_overriding.ipynb` file.

You can turn in the assignment multiple times, but only the last version you submitted will be graded.